# CS 5542 — Week 5 Snowflake Pipeline Notebook
This notebook is a **demo artifact**: connect → load → query → visualize.
Replace the example tables with your project subset tables.


## 1) Setup
- Copy `.env.example` to `.env` and fill in your Snowflake credentials.
- Install dependencies: `pip install -r requirements.txt`


In [ ]:
from snowflake.snowpark.context import get_active_session
import pandas as pd

session = get_active_session()
session

## 2) Sanity check: list tables


In [ ]:
df = session.sql("SHOW TABLES IN SCHEMA CS5542_WEEK5.PUBLIC").to_pandas()
df.head()


## 3) Query examples


In [ ]:
q = '''
SELECT TEAM, CATEGORY, COUNT(*) AS N
FROM CS5542_WEEK5.PUBLIC.EVENTS
GROUP BY TEAM, CATEGORY
ORDER BY N DESC
'''
session.sql(q).to_pandas()

## 4) Close connection


In [ ]:
# conn.close()
# 'closed'

# No need to close connection!

## 5) Convert CSV in DB schema


In [ ]:
CREATE OR REPLACE FILE FORMAT my_csv_format
TYPE = 'CSV'
SKIP_HEADER = 1
FIELD_OPTIONALLY_ENCLOSED_BY = '"';

## CSV File Loading

In [ ]:
import pandas as pd

df = pd.read_csv("aapl_balance_sheet(1).csv")
df.head()

df.info()
df.columns


In [ ]:
df = df.replace(',', '', regex=True)

for col in df.columns[1:]:   # skip first column if it's labels
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.head()

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()

In [ ]:
session.write_pandas(
    df,
    "AAPL_BALANCE_SHEET",
    auto_create_table=True
)

### SQL Queries in Pandas

In [ ]:
session.sql("SELECT * FROM AAPL_BALANCE_SHEET LIMIT 5").to_pandas()

In [ ]:
session.sql("SHOW COLUMNS IN TABLE AAPL_BALANCE_SHEET").to_pandas()

In [ ]:
session.sql("SELECT * FROM AAPL_BALANCE_SHEET LIMIT 10").to_pandas()


In [ ]:
session.sql("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'AAPL_BALANCE_SHEET'
""").to_pandas()

In [ ]:
session.sql("""
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'AAPL_BALANCE_SHEET'
ORDER BY ordinal_position
""").to_pandas()


df = session.sql("SELECT * FROM AAPL_BALANCE_SHEET").to_pandas()
df.head()


df.columns = (
    df.columns
      .str.strip()
      .str.upper()
      .str.replace(" ", "_")
      .str.replace(":", "")
)

df.columns

session.write_pandas(
    df,
    "AAPL_BALANCE_SHEET_CLEAN",
    auto_create_table=True,
    overwrite=True
)

session.sql("SELECT * FROM AAPL_BALANCE_SHEET_CLEAN LIMIT 5").to_pandas()

session.sql("""
SELECT
    NET_DEBT,
    TOTAL_DEBT,
    WORKING_CAPITAL
FROM AAPL_BALANCE_SHEET_CLEAN
LIMIT 10
""").to_pandas()

In [ ]:
session.sql("""
CREATE OR REPLACE VIEW AAPL_FINANCIAL_METRICS AS
SELECT
    NET_DEBT,
    TOTAL_DEBT,
    WORKING_CAPITAL,
    NET_DEBT / NULLIF(TOTAL_DEBT, 0) AS DEBT_RATIO
FROM AAPL_BALANCE_SHEET_CLEAN
""").collect()

In [ ]:
session.sql("SELECT * FROM AAPL_FINANCIAL_METRICS LIMIT 10").to_pandas()

In [ ]:
session.sql("""
CREATE OR REPLACE TABLE AAPL_BALANCE_NUMERIC AS
SELECT
    TRY_TO_NUMBER(REPLACE(NET_DEBT, ',', '')) AS NET_DEBT,
    TRY_TO_NUMBER(REPLACE(TOTAL_DEBT, ',', '')) AS TOTAL_DEBT,
    TRY_TO_NUMBER(REPLACE(WORKING_CAPITAL, ',', '')) AS WORKING_CAPITAL
FROM AAPL_BALANCE_SHEET_CLEAN
""").collect()

In [ ]:
session.sql("SELECT * FROM AAPL_BALANCE_NUMERIC LIMIT 5").to_pandas()

In [ ]:
df = session.sql("""
SELECT
    TRY_TO_NUMBER(REPLACE(NET_DEBT, ',', '')) AS NET_DEBT
FROM AAPL_BALANCE_SHEET_CLEAN
""").to_pandas()

import matplotlib.pyplot as plt

df.plot(kind='line')
plt.title("Net Debt Over Time")
plt.show()